In [1]:
import numpy as np
from pycalphad import Model, Database, calculate, equilibrium
import pycalphad.variables as v

#dbf = Database('2016-08-10-AlGdMgand18RLPSO-for 3d plot.tdb')
dbf = Database('alfe_sei.TDB')
models = {key: Model(dbf, ['AL', 'FE', 'VA'], key) for key in dbf.phases.keys()}

In [2]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
%load_ext cython
from Cython.Compiler.Options import _directive_defaults

_directive_defaults['linetrace'] = True
_directive_defaults['binding'] = True

/home/rotis/anaconda/envs/calphadpy3/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


In [29]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
cimport numpy as np
import numpy as np
from collections import defaultdict, OrderedDict
import copy
import itertools
import scipy.spatial
from pycalphad.core.sympydiff_utils import build_functions as compiled_build_functions
from pycalphad.core.constants import MIN_SITE_FRACTION, COMP_DIFFERENCE_TOL
from pycalphad.core.phase_rec cimport PhaseRecord, obj, grad, hess, mass_obj, mass_grad, mass_hess
from pycalphad.core.cache import cacheit
import pycalphad.variables as v
from pycalphad.core.eqsolver import _compute_phase_dof
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def _compute_constraints(object dbf, object comps, object phases,
                         object cur_conds, np.ndarray[dtype=np.float64_t, ndim=1] site_fracs,
                         np.ndarray[dtype=np.float64_t, ndim=1] phase_fracs, object phase_records):
    """
    Compute the constraint vector and constraint Jacobian matrix.
    """
    cdef int num_sitefrac_bals = sum([len(dbf.phases[i].sublattices) for i in phases])
    cdef int num_mass_bals = len([i for i in cur_conds.keys() if i.startswith('X_')]) + 1
    cdef double indep_sum = sum([float(val) for i, val in cur_conds.items() if i.startswith('X_')])
    cdef double[::1] comp_obj_value = np.atleast_1d(np.zeros(1))
    cdef object dependent_comp = set(comps) - set([i[2:] for i in cur_conds.keys() if i.startswith('X_')]) - {'VA'}
    dependent_comp = list(dependent_comp)[0]
    cdef int num_constraints = num_sitefrac_bals + num_mass_bals
    cdef int num_phases = len(phases)
    cdef int num_vars = len(site_fracs) + num_phases
    cdef np.ndarray[ndim=1, dtype=np.int_t] phase_dof = _compute_phase_dof(dbf, comps, phases)
    cdef np.ndarray[ndim=1, dtype=np.float64_t] l_constraints = np.zeros(num_constraints)
    cdef np.ndarray[ndim=2, dtype=np.float64_t] constraint_jac = np.zeros((num_constraints, num_vars))
    cdef np.ndarray[ndim=3, dtype=np.float64_t] constraint_hess = np.zeros((num_constraints, num_vars, num_vars), order='F')
    cdef double[::1] sfview
    cdef double[::1] comp_grad_value
    cdef double[::1,:] comp_hess_value
    cdef int phase_idx, var_offset, constraint_offset, var_idx, iter_idx, grad_idx, hess_idx, comp_idx, idx
    cdef PhaseRecord prn
    cdef double phase_frac

    # Ordering of constraints by row: sitefrac bal of each phase, then component mass balance
    # Ordering of constraints by column: site fractions of each phase, then phase fractions
    # First: Site fraction balance constraints
    var_idx = 0
    constraint_offset = 0
    for phase_idx in range(num_phases):
        name = phases[phase_idx]
        for idx in range(len(dbf.phases[name].sublattices)):
            active_in_subl = set(dbf.phases[name].constituents[idx]).intersection(comps)
            ais_len = len(active_in_subl)
            constraint_jac[constraint_offset + idx,
            var_idx:var_idx + ais_len] = 1
            l_constraints[constraint_offset + idx] = \
                (sum(site_fracs[var_idx:var_idx + ais_len]) - 1)
            var_idx += ais_len
        constraint_offset += len(dbf.phases[name].sublattices)
    # Second: Mass balance of each component
    for comp_idx, comp in enumerate(comps):
        if comp == 'VA':
            continue
        var_offset = 0
        phase_idx = 0
        for phase_idx in range(num_phases):
            name = phases[phase_idx]
            prn = phase_records[name]
            phase_frac = phase_fracs[phase_idx]
            spidx = len(site_fracs) + phase_idx
            sfview = site_fracs[var_offset:var_offset + phase_dof[phase_idx]]
            comp_hess_value = np.zeros((phase_dof[phase_idx], phase_dof[phase_idx]), order='F')
            comp_grad_value = np.zeros(phase_dof[phase_idx])
            with nogil:
                mass_obj(prn, comp_obj_value, sfview, comp_idx)
                mass_grad(prn, comp_grad_value, sfview, comp_idx)
                mass_hess(prn, comp_hess_value, sfview, comp_idx)
                # current phase frac times the comp_grad
                for grad_idx in range(var_offset, var_offset + phase_dof[phase_idx]):
                    constraint_jac[constraint_offset, grad_idx] = \
                        phase_frac * comp_grad_value[grad_idx - var_offset]
                    constraint_hess[constraint_offset, spidx, grad_idx] = comp_grad_value[grad_idx - var_offset]
                    constraint_hess[constraint_offset, grad_idx, spidx] = comp_grad_value[grad_idx - var_offset]
                    for hess_idx in range(var_offset, var_offset + phase_dof[phase_idx]):
                        constraint_hess[constraint_offset, grad_idx, hess_idx] = phase_frac * comp_hess_value[grad_idx - var_offset, hess_idx - var_offset]
                l_constraints[constraint_offset] += phase_frac * comp_obj_value[0]
                constraint_jac[constraint_offset, spidx] += comp_obj_value[0]
                var_offset += phase_dof[phase_idx]
        if comp != dependent_comp:
            l_constraints[constraint_offset] -= float(cur_conds['X_' + comp])
        else:
            # TODO: Assuming N=1 (fixed for dependent component)
            l_constraints[constraint_offset] -= (1 - indep_sum)
        constraint_offset += 1
    return l_constraints, constraint_jac, constraint_hess

In [30]:
%load_ext line_profiler
%lprun -f _compute_constraints equilibrium(dbf, ['AL', 'FE', 'VA'], ['B2_BCC', 'AL13FE4'], {v.T: 700, v.X('AL'): (0,1,0.02), v.P: 101325}, model=models, compute_constraints=_compute_constraints)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
